In [1]:
import pandas as pd
# import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from sklearn.metrics import precision_recall_fscore_support, accuracy_score



In [2]:
device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [3]:
import pandas as pd

# train = pd.read_csv('/home/kdt-admin/lsy/significant_clean1.csv')
train = pd.read_csv('/home/kdt-admin/lsy/preprocess2.csv')
# train=train[['clean1_result','malicious']]
# train=train[['hanspell','malicious']]
# train=train[['t5','malicious']]
# test = pd.read_csv('/home/kdt-admin/lsy/significant_clean1_test.csv')
# test = pd.read_csv('/home/kdt-admin/lsy/preprocess2_test.csv')
# test=test[['clean1_result','malicious']]
# test=test[['hanspell','malicious']]
train.sample(6)

,comments,malicious,clean1,hanspell,t5
13734,이런 틀딱 드립은 지양해라.. 그런건 탑골공원에서나 먹힌다,1,이런 틀딱 드립은 지양해라 그런건 탑골공원에서나 먹힌다,이런 틀딱 드립은 지양해라 그런 건 탑골공원에서 나 먹힌다,hate-speech 드립은 지양해라 그런 건 탑골 공원에서나 먹힌다.
4496,그래 잘한다 계속 칭찬해줘 잘하고있어,0,그래 잘한다 계속 칭찬해줘 잘하고있어,그래 잘한다 계속 칭찬해 줘 잘하고 있어,그래 잘한다 계속 칭찬해 줘 잘하고 있어.
6669,난 절대로 브래드피트랑은 안논다 @.@,1,난 절대로 브래드피트랑은 안논다,난 절대로 브래드 피트랑은 안 논다,난 절대로 브래드 피트랑은 안 논다.
25767,저표도 호남이 주작한거같은데?,1,저표도 호남이 주작한거같은데,지표도 호남이 주작한 거 같은데,저표도 호남이 주작한 거 같은데.
25412,뭔 레즈가 상처받노 또 또 레즈머리채 잡노 똥꼬충 후팔놈들이ㅡㅡ 지들 얘기에 아무 ...,1,뭔 레즈가 상처받노 또 또 레즈머리채 잡노 똥꼬충 후팔놈들이 지들 얘기에 아무 상관...,뭔 레즈가 상처받노 또 또 레즈 머리채 잡노 똥꼬충 후발 놈들이 자기들 얘기에 아무...,왜 레즈가 상처받냐 또 또 레즈 머리채 잡니 똥꼬충 후팔놈들이 자기들 얘기에 아무 ...
11289,"인맥자랑, 보여주기식 그이상 그이하도 아니다",1,인맥자랑 보여주기식 그이상 그이하도 아니다,인맥 자랑 보여주기식 그 이상 그 이하도 아니다,예전엔 인맥 자랑 보여 주기 그 이상 그 이하도 아니다.


In [4]:
# test.sample(6)

In [5]:
# import pandas as pd 
# from ast import literal_eval

# # 문자열을 리스트로 변환
# def str_to_list(x):
#     try:
#         if type(x) == str:
#         	return literal_eval(x)
#         elif type(x) == list:
#         	return x
#     except: #해당 값이 null값이거나 오류가 있을 때, None을 return 하기
#         return None
# train['clean1_result']=train['clean1_result'].apply(str_to_list)
# # test['clean1_result']=test['clean1_result'].apply(str_to_list)
# train['clean1_result'] = train['clean1_result'].apply(lambda x: ' '.join(x))
# # test['clean1_result'] = test['clean1_result'].apply(lambda x: ' '.join(x))
# train

In [6]:
# train['length'] = train['hanspell'].apply(lambda x: len(str(x)))
train['length'] = train['t5'].apply(lambda x: len(str(x)))
# test['length'] = test['clean1_result'].apply(lambda x: len(str(x)))
# train['length'] = train['clean1'].apply(lambda x: len(str(x)))
# test['length'] = test['clean1'].apply(lambda x: len(str(x)))
train = train.loc[train['length'] > 5]
# test = test.loc[test['length'] > 5]

In [7]:
train['malicious'].value_counts()

malicious
1    17642
0    10081
Name: count, dtype: int64

In [8]:

train = train.groupby('malicious').apply(lambda x: x.sample(n=9500,random_state=42)).reset_index(drop=True)
train['malicious'].value_counts()

/tmp/ipykernel_2218249/1209799692.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  train = train.groupby('malicious').apply(lambda x: x.sample(n=9500,random_state=42)).reset_index(drop=True)


malicious
0    9500
1    9500
Name: count, dtype: int64

In [9]:
# data_list = []

# for text, label in zip(train['clean1'], train['malicious']):
#   data = []
#   data.append(text.strip())
#   data.append(label)
#   data_list.append(data)

# test_data_list = []
# for text, label in zip(test['clean1'], test['malicious']):
#   data = []
#   data.append(text.strip())
#   data.append(label)
#   test_data_list.append(data)

In [10]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(train, test_size = 0.2, shuffle=True, random_state = 12)
# train, test = train_test_split(data_list, test_size = 0.2, shuffle=True, random_state = 12)

In [11]:
train.iloc[0]

comments     저외모에 운동했으니그동안 여선배 여코치들한테 얼마나 보적보 갈굼당했을지상상도 안간다 ㄹㅇ
malicious                                                    1
clean1       저외모에 운동했으니그동안 여선배 여코치들한테 얼마나 보적보 갈굼당했을지상상도 안간다...
hanspell     저 외모에 운동했으니 그동안 여자 선배여 코치들한테 얼마나 보적 보 갈굼 당했을지 ...
t5           저 외모에 운동했으니 그동안 여 선배 여 코치들한테 얼마나 보적보 갈굼 당했을지 상...
length                                                      57
Name: 9764, dtype: object

In [12]:
tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base")


In [13]:
class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts[index]
        label = self.labels[index]

        encoding = self.tokenizer(text, padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt')
        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()

        return {'input_ids': input_ids, 'attention_mask': attention_mask, 'label': label}

In [14]:
max_length = 64

labels = torch.tensor(list(train['malicious']), dtype=torch.long)
dataset = CustomDataset(list(train['clean1']), labels, tokenizer, max_length)

In [15]:
print(dataset[:5])
print(labels[:5])

{'input_ids': tensor([[  443,  7423,  1588,   266,  2566,  5652, 18412,   580,   469,  1085,
           580,  1592,   399,  2819,  1267,   447,   449,   496,  1422, 20378,
           371,  4861,   248,   400,  2485,  8537, 13258,   188,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3],
        [ 2748,   340,  1323, 15142,  2544,   647,  4157,  1724, 20444,  2748,
           468,  1300,  2005,   995,   188,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,


In [16]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(dataset, test_size=0.15, random_state=42)
batch_size=16
train_dataloader = DataLoader(train, batch_size=batch_size, shuffle=True)
valid_dataloader = DataLoader(test, batch_size=batch_size, shuffle=True)

In [17]:
model = AutoModelForSequenceClassification.from_pretrained("beomi/KcELECTRA-base", num_labels=2)
model.to(device)

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=3)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

In [18]:
# 하이퍼파라미터 설정
learning_rate = 1e-5
epochs = 5

# 옵티마이저 및 손실 함수 설정
optimizer = AdamW(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

/home/kdt-admin/miniconda3/envs/kobert/lib/python3.10/site-packages/transformers/optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [19]:
from tqdm import tqdm
import os
save_path = 'kcelectra_checkpoints/'
os.makedirs(save_path, exist_ok=True)
for epoch in range(epochs):
    model.train()
    total_loss = 0
    

    for batch in train_dataloader:
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['label']

        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        # 그래디언트 초기화
        optimizer.zero_grad()
        # 모델에 입력을 주어 예측을 생성합니다.
        outputs = model(input_ids, attention_mask=attention_mask)
        # 모델 출력에서 로짓(분류에 대한 점수)을 얻습니다.
        logits = outputs.logits
        # 손실을 계산합니다.
        loss = criterion(logits, labels)
        # 역전파를 통해 그래디언트 계산
        loss.backward()
        # 옵티마이저를 사용해 가중치를 업데이트
        optimizer.step()
        # 에포크 전체 손실을 누적합니다.
        total_loss += loss.item()

    # 에포크 평균 손실 계산
    avg_loss = total_loss / len(train_dataloader)
    # 에포크별 손실 출력
    print(f"Epoch {epoch+1}/{epochs} - Avg Loss: {avg_loss:.4f}")

    # 모델 평가
    model.eval()
    val_total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for val_batch in valid_dataloader:
            # Validation 데이터 가져오기
            val_input_ids = val_batch['input_ids']
            val_attention_mask = val_batch['attention_mask']
            val_labels = val_batch['label']

            val_input_ids = val_input_ids.to(device)
            val_attention_mask = val_attention_mask.to(device)
            val_labels = val_labels.to(device)

            # 모델 예측
            val_outputs = model(val_input_ids, attention_mask=val_attention_mask)
            val_logits = val_outputs.logits

            # 손실 계산
            val_loss = criterion(val_logits, val_labels)
            val_total_loss += val_loss.item()

            # 정확도 계산
            val_preds = val_logits.argmax(dim=1)
            correct += (val_preds == val_labels).sum().item()
            total += val_labels.size(0)
    val_avg_loss = val_total_loss / len(valid_dataloader)
    val_accuracy = correct / total
    print(f"Validation Loss: {val_avg_loss:.4f} - Validation Accuracy: {val_accuracy:.4f}")
    precision, recall, fscore, _ = precision_recall_fscore_support(val_labels.cpu(), val_preds.cpu(), average='binary')
    print(f'precision: {precision}, recall: {recall}, fscore: {fscore}')
    torch.save(model.state_dict(), f'{save_path}model_epoch_{epoch+1}.pt')

Epoch 1/5 - Avg Loss: 0.4847
Validation Loss: 0.3904 - Validation Accuracy: 0.8298
precision: 1.0, recall: 0.6666666666666666, fscore: 0.8
Epoch 2/5 - Avg Loss: 0.3411
Validation Loss: 0.3702 - Validation Accuracy: 0.8338
precision: 1.0, recall: 1.0, fscore: 1.0
Epoch 3/5 - Avg Loss: 0.2636
Validation Loss: 0.4127 - Validation Accuracy: 0.8386
precision: 1.0, recall: 0.6, fscore: 0.75
Epoch 4/5 - Avg Loss: 0.1872
Validation Loss: 0.4695 - Validation Accuracy: 0.8307
precision: 1.0, recall: 0.6666666666666666, fscore: 0.8
Epoch 5/5 - Avg Loss: 0.1219
Validation Loss: 0.5320 - Validation Accuracy: 0.8430
precision: 1.0, recall: 0.8571428571428571, fscore: 0.9230769230769231
